# 1. Aufteilen der Datensätze in Test und Train
Alle Bilder und xml-Datein sollten im Ordner C:\Tensorflow2\workspace\training_demo\images   abgelegt sein
ebenso sollte unterordner 'test' und 'train' angelegt sein.  

In [ ]:
import os
import re
from shutil import copyfile
import math
import random
import time

imageDir = r'C:\Tensorflow2\workspace\Training_demo\images'
outputDir = r'C:\Tensorflow2\workspace\Training_demo\images'
ratio = 0.1
xml = True

def iterate_dir(source, dest, ratio, copy_xml):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)

    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        filename = images[idx]
        copyfile(os.path.join(source, filename),
                 os.path.join(test_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

    for filename in images:
        copyfile(os.path.join(source, filename),
                 os.path.join(train_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(train_dir, xml_filename))


def main():   

    # Now we are ready to start the iteration
    start_time = time.perf_counter()
    print('Start')
    iterate_dir(imageDir, outputDir, ratio, xml)
    end_time = time.perf_counter() - start_time
    print('Ende: Dauer = ', end_time)

if __name__ == '__main__':
    main()

# 1b. Andere Label verwenden (optional)
neue xml-Datein zu den bereits aufgeteilten Bildern zuordnen
dazu die neuen xml-Dateien C:\Tensorflow2\workspace\training_demo\images  ablegen
unötig zu sagen dass die xml-Dateien den gleichen Namen haben wie das dazugehörige Bild

In [ ]:
#Test_Train_xml.py
# wird nur benötigt wenn Labels geändert wurden und neue XML-Datein zu den bereits aufgeteileten 
# Bilder zugordenet werden sollen 
import os
import glob
import time

imageDir = r'C:\Tensorflow2\workspace\Training_demo\images'
#CWD_Path = os.getcwd()
start_time = time.perf_counter()
print('Start:')
for Folder in ['Test','Train']:
    print(Folder)
    for img in glob.glob(os.path.join(imageDir,Folder,'*.jpg')):
        img_path = os.path.split(img)
        img_path_2 = img_path[1].split('.')

        for xml_file in glob.glob(CWD_Path + '\*.xml'):
            xml_path = os.path.split(xml_file)
            xml_path_2 = xml_path[1].split('.')

            if img_path_2 [0] == xml_path_2[0]:
                print(img_path[1])
                print(xml_path[1])
                os.replace(xml_file, os.path.join(CWD_Path,Folder,xml_path[1]))
                
end_time = time.perf_counter() - start_time
print('Ende: Dauer = ', end_time)
print('Daten wurden aufgeteilt.')

# 2. Label_Map erstellen
die Namen der Labels müssen angepasst werden.

In [ ]:
labels = [{'name':'Label1', 'id':1},{'name':'Label2', 'id':2},{'name':'Label3', 'id':3},{'name':'Label4', 'id':4}]

with open(r'C:\Tensorflow2\workspace\Training_demo\annotations\label_map1.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')
print('Label_map.pbtxt wurde erstellt.')

# 3a. Umwandeln der csv in tfrecord
# Test
erstellt eine csv und eine .tfrecord Datei für Test
Achtung!! es muss eine label_map.pbtxt vorhanden sein unter C:\Tensorflow2\workspace\training_demo\annotations\

In [ ]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

xml_dir = r'C:\Tensorflow2\workspace\Training_demo\images\test'
labels_path = r'C:\Tensorflow2\workspace\Training_demo\annotations\label_map.pbtxt'
output_path = r'C:\Tensorflow2\workspace\Training_demo\annotations\test.record'
image_dir = xml_dir
csv_path = r'C:\Tensorflow2\workspace\Training_demo\images\_Test_labels.csv'

label_map = label_map_util.load_labelmap(labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)


def xml_to_csv(path): 
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    start_time = time.perf_counter()
    print('Start:')
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = xml_to_csv(xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        #print(group['filename'])
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))
    if csv_path is not None:
        examples.to_csv(csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(csv_path))
        
    end_time = time.perf_counter() - start_time
    print('Ende: Dauer = ', end_time)

if __name__ == '__main__':
    tf.app.run()


# 3b. Umwandeln der csv in tfrecord
# Train
erstellt eine csv und eine .tfrecord Datei für Train

In [ ]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

xml_dir = r'C:\Tensorflow2\workspace\Training_demo\images\train'
labels_path = r'C:\Tensorflow2\workspace\Training_demo\annotations\label_map.pbtxt'
output_path = r'C:\Tensorflow2\workspace\Training_demo\annotations\train.record'
image_dir = xml_dir
csv_path = r'C:\Tensorflow2\workspace\Training_demo\images\_Train_labels.csv'

label_map = label_map_util.load_labelmap(labels_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)


def xml_to_csv(path): 
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    start_time = time.perf_counter()
    print('Start:')
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = xml_to_csv(xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        #print(group['filename'])
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))
    if csv_path is not None:
        examples.to_csv(csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(csv_path))
        
    end_time = time.perf_counter() - start_time
    print('Ende: Dauer = ', end_time)

if __name__ == '__main__':
    tf.app.run()


# 4. pipeline config anpassen
dazu die piplien.config vom pretrained model kopieren in 
und Anpassungen bei folgenden Punkten vornehmen:


# 4. Das Training
individuell angepasst werden müssen die Zeilen 9,10,11 evt. 19 

vorher muss ein pretrained Model von https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md runtergelade und in C:/Tensorflow2/workspace/training_demo/prtrained_Models  gespeichert werden.
In diesem Bsp. faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8   genutzt.

Von da die entsprechende config -Datei  nehmen und in C:/Tensorflow2/workspace/training_demo/models/my_model...   speichern
in der verschobenen config-Datei müssen die Zeilen 
110 - fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
143 - label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
145 - input_path: "PATH_TO_BE_CONFIGURED/train2017-?????-of-00256.tfrecord"
156 - label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
160 - input_path: "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"
angepast werden.



In [3]:
# model_main_tf2.py
# individuell angepasst werden müssen die Zeilen 23, 24

#from absl import flags
import os
import tensorflow.compat.v2 as tf
from object_detection import model_lib_v2
import time

Path_to_scripts = r'C:\Tensorflow2\workspace\Training_demo'
model_dir = os.path.join(Path_to_scripts, 'models\my_faster_rcnn_resnet50_v1')
pipeline_config_path = os.path.join(model_dir, 'pipeline.config')
num_train_steps = None
eval_on_train_data = False
sample_1_of_n_eval_examples = None
sample_1_of_n_eval_on_train_examples = 5
checkpoint_dir = None
eval_timeout = 3600
use_tpu = False
num_train_steps = 200000
num_workers = 1
checkpoint_every_n = 1000
record_summaries = True

def main():  

  if checkpoint_dir:
    model_lib_v2.eval_continuously(
        pipeline_config_path=pipeline_config_path,
        model_dir=model_dir,
        train_steps=num_train_steps,
        sample_1_of_n_eval_examples=sample_1_of_n_eval_examples,
        sample_1_of_n_eval_on_train_examples=(
            sample_1_of_n_eval_on_train_examples),
        checkpoint_dir=checkpoint_dir,
        wait_interval=300, timeout=eval_timeout)
  else:
    if use_tpu:
      # TPU is automatically inferred if tpu_name is None and
      # we are running under cloud ai-platform.
      resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
          tpu_name)
      tf.config.experimental_connect_to_cluster(resolver)
      tf.tpu.experimental.initialize_tpu_system(resolver)
      strategy = tf.distribute.experimental.TPUStrategy(resolver)
    elif num_workers > 1:
      strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    else:
      strategy = tf.compat.v2.distribute.MirroredStrategy()

    with strategy.scope():
      model_lib_v2.train_loop(
          pipeline_config_path=pipeline_config_path,
          model_dir=model_dir,
          train_steps=num_train_steps,
          use_tpu=use_tpu,
          checkpoint_every_n=checkpoint_every_n,
          record_summaries=record_summaries)

if __name__ == '__main__':
    start_time = time.perf_counter()
    main()
    end_time = time.perf_counter() - start_time
    print('Training abgeschlossen: Dauer = ', end_time)
  #tf.compat.v1.app.run()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


[06/01 21:50:32] tensorflow INFO: Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Maybe overwriting train_steps: 200000


[06/01 21:50:32] tensorflow INFO: Maybe overwriting train_steps: 200000


INFO:tensorflow:Maybe overwriting use_bfloat16: False


...

INFO:tensorflow:Step 199900 per-step time 0.161s loss=0.060


[06/01 22:44:45] tensorflow INFO: Step 199900 per-step time 0.161s loss=0.060


INFO:tensorflow:Step 200000 per-step time 0.161s loss=0.219


[06/01 22:45:01] tensorflow INFO: Step 200000 per-step time 0.161s loss=0.219


Training abgeschlossen: Dauer =  3269.5293088999997


# Evaluate the Training
anders als beim Training ist hier ein Wert für Checkpoin_dir gesetzt.
Zeile 21: checkpoint_dir = model_dir
unvorteilhafter weise kann die Evaluierung nicht parallel zum Training laufen (kommt hoffentlich noch, bei TF1 gings ja auch). Also muss das Training mit num_train_steps = xxx  in kleine Portionen geteilt werden, und zwischendurch immer eval- wer es bracuht.

In [ ]:
# model_main_tf2.py
# individuell angepasst werden müssen die Zeilen 23, 24

#from absl import flags
import os
import tensorflow.compat.v2 as tf
from object_detection import model_lib_v2
import time

Path_to_scripts = r'C:\Tensorflow2\workspace\Training_demo'
model_dir = os.path.join(Path_to_scripts, 'models\my_faster_rcnn_resnet50_v1')
pipeline_config_path = os.path.join(model_dir, 'pipeline.config')
num_train_steps = None
eval_on_train_data = False
sample_1_of_n_eval_examples = None
sample_1_of_n_eval_on_train_examples = 5
checkpoint_dir = model_dir
eval_timeout = 180
use_tpu = False
num_train_steps = None
num_workers = 1
checkpoint_every_n = 1000
record_summaries = True

def main():  

  if checkpoint_dir:
    model_lib_v2.eval_continuously(
        pipeline_config_path=pipeline_config_path,
        model_dir=model_dir,
        train_steps=num_train_steps,
        sample_1_of_n_eval_examples=sample_1_of_n_eval_examples,
        sample_1_of_n_eval_on_train_examples=(
            sample_1_of_n_eval_on_train_examples),
        checkpoint_dir=checkpoint_dir,
        wait_interval=300, timeout=eval_timeout)
  else:
    if use_tpu:
      # TPU is automatically inferred if tpu_name is None and
      # we are running under cloud ai-platform.
      resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
          tpu_name)
      tf.config.experimental_connect_to_cluster(resolver)
      tf.tpu.experimental.initialize_tpu_system(resolver)
      strategy = tf.distribute.experimental.TPUStrategy(resolver)
    elif num_workers > 1:
      strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    else:
      strategy = tf.compat.v2.distribute.MirroredStrategy()

    with strategy.scope():
      model_lib_v2.train_loop(
          pipeline_config_path=pipeline_config_path,
          model_dir=model_dir,
          train_steps=num_train_steps,
          use_tpu=use_tpu,
          checkpoint_every_n=checkpoint_every_n,
          record_summaries=record_summaries)

if __name__ == '__main__':
    start_time = time.perf_counter()
    main()
    end_time = time.perf_counter() - start_time
    print('Evaluierung abgeschlossen: Dauer = ', end_time)
  #tf.compat.v1.app.run()


[06/01 22:50:38] tensorflow WARNING: Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None


[06/01 22:50:38] tensorflow INFO: Maybe overwriting sample_1_of_n_eval_examples: None


INFO:tensorflow:Maybe overwriting use_bfloat16: False


[06/01 22:50:38] tensorflow INFO: Maybe overwriting use_bfloat16: False


...

[06/01 22:51:10] tensorflow INFO: 	+ Loss/regularization_loss: 0.000000


INFO:tensorflow:	+ Loss/total_loss: 0.353469


[06/01 22:51:10] tensorflow INFO: 	+ Loss/total_loss: 0.353469


INFO:tensorflow:Waiting for new checkpoint at C:\Tensorflow2\workspace\Training_demo\models\my_faster_rcnn_resnet50_v1


[06/01 22:55:38] tensorflow INFO: Waiting for new checkpoint at C:\Tensorflow2\workspace\Training_demo\models\my_faster_rcnn_resnet50_v1


# 5. Ergebnisse in Tensorboard
dazu die Commands in Anacoda-prompt  oder cmd ausführen.
command3 - evt. muss der Ordner my_....   angepasst werden 

In [ ]:
command1 = "activate tf2_env" 
command2 = "cd C:\Tensorflow2\workspace\Training_demo"
command3 = "tensorboard --logdir=models/my_faster_rcnn_resnet50_v1"
print(command1)
print(command2)
print(command3)

# 5. export model
im command müssen die beiden Pfad angepasst werden:
pipelin_config_path
trained_checkpoint_dir

und die exporter_main_v2.py sollte in  C:\Tensorflow2\workspace\training_demo\  liegen
zu finden ist sie unter: C:\Tensorflow2\models-master\research\object_detection

In [ ]:
command = python C:\Tensorflow2\workspace\Training_demo\exporter_main_v2.py --input_type image_tensor --pipeline_config_path C:\Tensorflow2\workspace\training_demo\models\my_faster_rcnn_resnet50_v1\pipeline.config --trained_checkpoint_dir C:\Tensorflow2\workspace\training_demo\models\my_faster_rcnn_resnet50_v1\ --output_directory C:\Tensorflow2\workspace\training_demo\exported-models\my_model

In [ ]:
!{command}

# 6. Run model with images

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# %%
# This demo will take you through the steps of running an "out-of-the-box" TensorFlow 2 compatible
# detection model on a collection of images. More specifically, in this example we will be using
# the `Saved Model Format <https://www.tensorflow.org/guide/saved_model>`__ to load the model.

# %%
# Download the test images
# ~~~~~~~~~~~~~~~~~~~~~~~~
# First we will download the images that we will use throughout this tutorial. The code snippet
# shown bellow will download the test images from the `TensorFlow Model Garden <https://github.com/tensorflow/models/tree/master/research/object_detection/test_images>`_
# and save them inside the ``data/images`` folder.
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
#import pathlib
import tensorflow as tf
import time
import numpy as np
import glob
import cv2
#from PIL import Image
#import matplotlib.pyplot as plt
import warnings
from utils import label_map_util
#from utils import visualization_utils as viz_utils

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


IMAGE_PATHS = r'C:\Tensorflow2\workspace\training_demo\test'
PATH_TO_LABELS = r'C:\Tensorflow2\workspace\training_demo\annotations\label_map.pbtxt'
PATH_TO_MODEL_DIR = r'C:\Tensorflow2\workspace\training_demo\exported-models\my_model'
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
alpha = 0.1 #transparency factor
prediction_score = 0.5 #treshold  for prediction
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

# Here are some simple things to try out if you are curious:
#
# * Modify some of the input images and see if detection still works. Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
# * Print out `detections['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
# * Set ``min_score_thresh`` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

images = glob.glob(os.path.join(IMAGE_PATHS, '*.jpg'))

for img in images:

    #print('Running inference for {}... '.format(img), end='')

    image = cv2.imread(img)  # (PATH_TO_IMAGE)
    # Things to try:
    #image = cv2.flip(image, 1)  # horizontal flip
    #image = cv2.flip(image, 0)  # vertical flip

    image_copy = cv2.resize(image, (1280, 720))
    overlay = image_copy.copy()

    image_np = np.array(image)#load_image_into_numpy_array(image_path)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    #image_np_with_detections = image_np.copy()

    for i in range(0, len(detections['detection_classes'])):
        if detections['detection_scores'][i] > prediction_score:
            ymin, xmin, ymax, xmax = detections['detection_boxes'][i]
            class_name = category_index[detections['detection_classes'][i]]['name']
            xmin = int(xmin * 1280)
            xmax = int(xmax * 1280)
            ymin = int(ymin * 720)
            ymax = int(ymax * 720)
            score = round(detections['detection_scores'][i] * 100, 1)
            print(xmin, ymin, xmax, ymax, class_name, score)
            text = class_name + ' ' + str(score)
            cv2.rectangle(overlay, (xmin, ymin), (xmax, ymax), [0, 0, 255], -1)
            cv2.rectangle(image_copy, (xmin, ymin), (xmax, ymax), [255, 255, 255], 2)
            image = cv2.putText(image_copy, text, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                                cv2.LINE_AA)

    image_copy = cv2.addWeighted(overlay, alpha, image_copy, 1 - alpha, 0)
    cv2.imshow('result', image_copy)

    # Press any key to close the image
    k = cv2.waitKey(0)

cv2.destroyAllWindows()

# sphinx_gallery_thumbnail_number = 2